# For everyone

We thank the reviewers for their careful reading of our manuscript and their insightful feedback. We have provided detailed responses to each comment. Still, we would like to highlight in particular the following points.

Several observations addressed comparisons with complete branch-and-bound verifiers such as $\beta$-CROWN based Branch and Bound, or Marabou. Our work introduces an incomplete verifier, so our primary comparisons focus on methods aimed at producing lower bounds on the certification objective.  

Concerning the experimental set-up, to the best of our knowledge, SDP-based methods have not yet been deployed on modern CNN architectures such as ResNets, given the challenge of scaling to millions of parameters and handling residual connections and normalization layers. Although advancing the scalability of SDP-based models is an important and exciting research direction, it remains orthogonal to our current contributions.

# Review 1

*The literature review is generally restricted to a single paper from 2022 and the rest being older than that. There is quite a bit of more recent work, esp. https://arxiv.org/abs/2506.06665, which has very much improved the state of the art of relaxations to SDP for NN verification. Oddly, this paper appears in the reference list, but is not referred to in the text. So, I am a bit concerned that this and other recent work is not compared against.* 

We would like to note that this reference appears in our bibliography as we refer to it in our extended related work (see Supplementary Material, Section D.1), where we provide a more extensive overview of the state of the art of SDP-based certification methods. We did not compare our approach with the SDP-CROWN method since SDP-CROWN was developed in the context of $\ell_2$​-norm adversaries while in this paper we focus on $\ell_\infty$​-norm adversaries. 
	
*Some parts of the model are unclear. For example, it is not clear whether or not the beta (binary) variables are relaxed or not. From the notation, I think the beta variables remain binary. However, my understanding is that Mosek does not support mixed integer SDPs. Either way, I have a couple questions below.*

Thank you for raising this point. We would like to clarify that our SDP relaxation does include the relaxation of binary variables. This is induced by constraint (21) (i.e., z² = z). Therefore, the numerical results reported do not include a B&B on the binary variables. Note that the introduction of RLT cuts significantly reinforces this relaxation. Our approach extends directly to solving SDPs with binary variables, Since no SDP solver is designed to solve SDPs with binary variables, implementing a dedicated B&B could be interesting for future work. While providing a better quality bound, this would also greatly increase the computation time, thus penalizing scaling.

### Questions

*Are the binary variables relaxed or binary in the final relaxation?*

Yes, the binary variables are relaxed in (SDP).

*If relaxed, I would expect the relaxation to be weaker than the enumeration of all classes using SDP with the cuts that are specific to each class. (because the optimization can assign fractional values to beta).*

You are absolutely right, if we were to compare our method with a targeted formulation computed separately for each class and without any RLT cuts, our approach would likely appear weaker.
However, this relationship between the optimal values of the two problems is no longer true as we are able to incorporate substantially more RLT cuts in (SDP_u) than in (SDP_t). The values of the bounds are then no longer comparable.


*Is this the case? Do you have CPU and solution quality comparisons between your model enumerated per class vs. your one large model? I am not completely sure if SDP_t is exactly this or not.*

In terms of solution quality for models with RLT, we observe in the numerical results that the bounds obtained with (SDP_u) are tighter than those obtained with (SDP_t), since the number of data certified by (SDP_u) is significantly greater than that certified by (SDP_t).
In terms of CPU time, since the two models have approximately the same size, solving (SDP_u) or (SDP_t) on one target requires a similar computing time. Thus, our new approach is significantly faster since it requires only a single resolution (that of (SDP)) to certify a data point, whereas with targeted approaches, it is necessary to resolve the model (SDP_t) as many times as there are classes in the network. The computation time of our new approach is therefore reduced by a factor equal to the number of classes in the network.
It should also be noted that this single resolution allows us to add a very large number of RLT cuts in (SDP) and to obtain (experimentally) a bound that is very often tighter than that obtained by the targeted method. 

*If not relaxed, is the main value of the combined model that it can take advantage of the branching strategies and pruning capabilities of the mixed integer solver, and not have to do a complete enumeration of all classes (e.g., the search tree is essentially one layer below the root node, with each leaf essentially corresponding to one verification class and many leaves pruned and never evaluated)?*

Yes, you are absolutely right. Developing a B&B based on the bound obtained with (SDP_u) to calculate the solution in binary variables would further improve the value of the bound. However, there is no mixed-SDP solver, and this would require us to develop our own B&B, which would not benefit from the highly effective heuristics developed for binary variable optimization that have been introduced in commercial solvers over the last 20 years. This extension is a future work. 
It is important to note that applying this B&B could significantly increase computation time, and a good balance will indeed need to be reached between improving the bound and scaling.
We did, however, test submitting (QP) to the commercial solver Gurobi, and the results showed that directly solving the quadratic formulation is too slow and does not scale.



# Review 2

*The experimental evaluation on $\beta$-CROWN seems unfair since the verifier is only run for very short time budgets (2-5 seconds) while the authors' proposed method is run for up to ~1600 seconds. Since  $\beta$-CROWN is a complete verifier, a fair comparison would enable its branching and run it for the same time budget as . The fact that the numbers reported here are not representative is also evidenced by comparing them to e.g. those reported by [6] where the gap between  \beta-CROWN and the SDP verifiers is significantly smaller than the gap reported in this work.*

In our work, we propose an incomplete verifier *i.e.* we are only interested in computing a lower bound for the objective. Thus, we are comparing the performances against other incomplete verifiers. We then only reported the performances of $\beta$-CROWN used to calculate lower and upper bounds for each neuron. The running time of $\beta$-CROWN corresponds thus to the required time to compute those bounds.
However, we do agree that running a branch and bound search with the same time budget as our method would be an interesting comparison between an incomplete and a complete verifier. To further assess our approach performances, we ran our instances with $\beta$-CROWN combined with branch and bound. Interestingly, our approach is able to certify better in medium to large perturbation regimes (*c.f.* Table below).


| Model | ε = 0.026 | ε = 0.05 | ε = 0.01|
|--------|---------------|----------------|---------------|
| SDP_u  |      86% - 323s         |    84% - 1391s            |    62% - 3820s            |
| SDP_t  |      74% - 399s         |    6% - 3977s            |    0% - 8890s            |

We would be glad to add those results in the final version. 


We thank the reviewer for bringing these additional references to our attention. We will cite them in the state of the art section of our final version. 

While interesting, the suggested reference [2] was published late after ICLR submission deadline (10/26/25), so we did not compare ourselves to it. We did, however, test submitting (QP) to the commercial solver Gurobi, and the results showed that directly solving the quadratic formulation is too slow and does not scale even for the smallest network considered in our experiences : 6x100 (timeout = 3h). 
We appreciate the reviewer’s observation that our method can be viewed as a generalization of [10]. However, to the best of our knowledge, no implementation of [10] is publicly available, which prevents a direct empirical comparison.

We  also will discuss GCP-CROWN in our extended related work. Our focus in the paper is primarily on improving and scaling SDP-based verifiers. We, thus, only compare our new approach with SDP-based methods and other well known incomplete verifiers. The code of methods proposed in [6,7,9] have to the best of our knowledge not been published. Regarding the method in [8], we found an undocumented Matlab library that we were unable to run and might not be trivially compatible with most deeplearning python libraries.

To the best of our knowledge no SDP relaxation is able to scale with ResNet and large CNN. This would imply scaling SDP matrices to a large number of parameters by taking into account convolutional kernel induced sparsity, to deal with residual connections and batch-norm. While this is indeed a very exciting line of work, it is orthogonal to our contributions.

All details of the networks we train are reported in section C of the appendices, we will add a pointer in section 5.4 to the appendix.
The network is a feed forward network of sizes 784x6x100 and 6x100-N with N being the total number of classes : $N \in [5,67]$. We trained it with during 100 epochs and obtain a final accuracy score of 75.2%.

We thank you for your careful reading of the manuscript. We rewrote the sentence “However, unmodeled quadratic terms appear in Constraint (5), i.e..” in the final version as : “The removal of active neurons from the network leads to new cross-layer dependencies that are modeled by quadratic terms not occurring in the chordal decomposition. These unmodeled quadratic terms appear in Constraint (5), i.e. …”. We have also corrected the typos you pointed out.

# Review 3


Regarding the RLT cuts, this is indeed an important point, and we appreciate the opportunity to clarify it. We use the same heuristic as [Lan, 2022], where the model (SDP_t) was introduced, to select the RLT cuts. It is clear that the larger the parameter p, the better the bound will be. An important advantage of our new formulation (SDP) is that it allows integrating up to 100% of the RLT cuts (for 6x100 and 9x100 networks), whereas (SDP_t) can only handle 20% of these cuts for the same networks. As a rule of thumb, we take as much RLT cuts as the solver can handle ($\simeq  50 000$).

To further assess our approach, we have conducted new tests on the 6×100 network that illustrate the impact of varying the parameter p on computation time and certification percentage (see the following table). These results show that, unlike (SDP_t), the computation time does not blow up for (SDP_u) when p approaches 100%, and that the certification percentage is significantly better when p equals 100%.

| Model | p=0% | p=10% | p=30% | p = 60% | p = 100% |
|--------|---------------|----------------|---------------|----------------|---------------|
| SDP_u  | 70% - 11s | 72% - 35s  |   73% - 197s  | 73% - 304s   |   86% - 323s        |

| Model | p=0% | p=10% | p=20% |  p = 100% |
|--------|---------------|----------------|---------------|---------------|
| SDP_t  | 72% - 22s | 74% - 399s  |   75% - 1250s  |   77% - 2522s        |



To the best of our knowledge no SDP relaxation is able to scale with ResNet and large CNN. Adapting SDP-based certification methods to leverage the sparse structure of convolutional layers, residual connections, pooling and batch-norm is an exciting line of work but is orthogonal to our contributions. 

We agree that a comparison with incomplete verifiers is needed to assess the performances of our method. In Table 1, we reported results of 3 SDP-based incomplete verifiers, and $\beta$-Crown. Furthermore, regarding the LP bound, we know that LP relaxations lead to weaker bounds than SDP ones.

Note that we reported in the appendix the fraction of eliminated classes vs. network size (see the implementation details part C, Table 4). For each number of classes, the number of “running targets” corresponds to the average number of targets that could not be pruned by beta-CROWN. 

(i) The fraction of verified samples increases significantly when 50–70% of neurons are stable; the effect diminishes once stability is very high.
(ii) We evaluated the impact of pruning active neurons (see Q2).
(iii) Typical failures appear to be due to numerical issues on very small perturbations; we could not identify a consistent pattern (see Q2).

### Questions

*(1)*

Adding more RLT cuts strengthens the bound. However, we are limited by the capacity of the SDP solver we use, which can handle, within a reasonable time, SDP problems with approximately 50,000 constraints. Therefore, depending on the network size, we select at most 250,000 constraints generated by the heuristic proposed in [Lan, 2022].

*(2)*

Pruning active neurons introduces new cross-layer quadratic dependencies not covered by the chordal decomposition; these terms are relaxed through Constraints (28)–(31), giving weaker bounds. However, pruning yields much smaller SDPs, enabling integration of up to 100% of the RLT cuts, which compensates for the relaxation. The resulting formulations use different cut sets and are not directly comparable in terms of bound quality.
To quantify the impact of neuron pruning, we define the gap as the relative difference between the lower bound obtained after pruning and the original lower bound without pruning.
We evaluated the bound gap (pruned – unpruned) on 6×100 and 6×200. The gaps remain small, indicating that certification is robust to active-neuron pruning:


| Network | Min Gap | Max Gap | Mean Gap |  Median Gap |
|--------|---------------|----------------|---------------|---------------|
| 6x100 | -0.90| -7.73  |   +4.11  |  -0.38       |
| 6x200 | -0.27 | -3.95  |   +4.32  |  +0.01      |

*(3)*

We will include the full curve in the final version. Below are the numerical results, showing that β-CROWN pruning is fast but certifies < 20% of neurons:

| Method | 5 classes | 20 classes | 50 classes |  67 classes |
|--------|---------------|----------------|---------------|---------------|
|  SDP_u | 80% - 959s| 70% - 1139s  |  98% - 1297s  |  90% - 1829s      |
| SDP_t| 40% - 563s | 25% - 4945s  |  4% - 14622s  |   7% - 16410s   |
| $\beta$-CROWN| 20% - 3s | 20% - 3s  |  4% - 2s  | 4% - 3s |

*(4)*

Shallow fully connected neural networks on CIFAR10 show poor accuracy (less than 40%). In this set-up, decision boundary of the network might to chaotic to obtain meaningful certification results.  

# Review 4


We thank the reviewer for his insightful remarks. We followed the same validation protocol as in [Lan 2022] which is standard for SDP-based certification methods. To further assess our approach, we varied perturbation sizes on neural network 6x100 and obtain the following results:

| Model | ε = 0.026 | ε = 0.05 | ε = 0.01|
|--------|---------------|----------------|---------------|
| SDP_u  |      86% - 323s         |    84% - 1391s            |    62% - 3820s            |
| SDP_t  |      74% - 399s         |    6% - 3977s            |    0% - 8890s            |


Overall our method remains good in verification percentage, even for great epsilons, while current untargeted models do not scale to great perturbation sizes.
Furthermore, we analyzed the (SDP) model with RLT fixed at 50% with several values of epsilon (see the following table) for a larger network with 8 hidden layers of size 1024. The results reveal the expected trend: the larger the network and the perturbation, the lower is the number of constraints when full pruning is applied.

|  ε | Pruning| Nb constraints | Nb variables |
|--------|---------------|----------------|---------------|
| 0.01 |    Partial      |    7 484 566            |    17 364 036            |
| 0.01  |     Full    |   7 082 923          |    16 805 075         |
| 0.1  |     Partial   |   12 299 601          |    32 548 142       |
| 0.1  |     Full    |   12 299 621         |    32 548 069      |


We thank the reviewer for his careful reading. We have carefully checked the final version and corrected the final version. As suggested, we also reorganize the presentation of the results to clarify the contribution. We have corrected the consistency of the writing and the double notation for the dataset. We also removed the notation K. We added a reference to Figure 1 in the final version. We agree that Property 3 and Theorem 1 are explained in words before being formally written, however to facilitate understanding we choose to keep Property 3 and Theorem 1 in the final version.  

### Questions

*(1)* Fully connected neural networks on CIFAR10 have sometimes been used ([Lan, 2023]), however they do not reach a good clean accuracy (less than 40%). Their performances are thus too erratic to be certified.

*(2)* We give here the main ideas : By denoting n^k_a the number of stable active neurons on layer k, and n^k_u the number of unstable neurons, when applying the chordal decomposition each block matrix P_k (k=0 .. K-3) is a symmetric matrix of dimension (1+ n^k_u  + n^k_a +1+ n^{k+1}_u  + n^{k+1}_a), by removing the stable neurons theses sizes become (1+ n^k_u  +1+ n^{k+1}_u ). Now for the matrix P_{K-2} that is composed of elements z and of beta (with a number |J_K|) , the new size goes from  (1+ n^{K-1}_u  + n^{K-1}_a +1+ n^{K}_u  + n^{K}_a + |J_K|) to  (1+ n^{K-1}_u +1+ n^{K}_u + |J_K|). We will add this explanation in the appendix to clarify the proposition.

*(3)* The percentages represent the verification percentage obtained on the data run on. We see that our model is able to certify a significant amount of data even when the number of classes of the dataset increases. For the 50 classes and 67 classes points in our graph, we initially ran (SDP_t) only on 5 data due to the high resolution time. We ran it thoroughly and finally obtained 4% and 7% of certification respectively. 

*(4)* Our contribution is divided into three parts : (i) an exact untargeted quadratic formulation and its SDP relaxation, (ii) tightening cuts, and (iii) a stable active neuron ablation. We would like to highlight that SDP_u corresponds to the untargeted objective applied to the model  (SDP_t), for which we add additional cuts.  
We also ran our model SDPu without any RLT cuts, which is the application of the  untargeted objective to (SDP_Layer) on network 6x100, and we get a certification percentage of 70% with a mean computing time of  11s.

*(5)* Thank you for raising this point. In other approaches from literature, the choice of an adversarial target has to be made which might bias the results. However, in our paper, we run all models to certify across all targets. At the end, for each targeted method, the number of runs is equal to the number of evaluated data times the number of classes.

*(6)* We have conducted experiments to evaluate the percentage of verified data as a function of the number of stable neurons. We observe that when the proportion of active stable neurons ranges between 50% and 70%, the number of certified data points is significantly higher when pruning the active neurons. For larger proportions, pruning degrades the certification. The study of the impact magnitude is shown above (see answer to question 1).
Finally, experiments in Tab. 1 show a global trend with respect to the network depth. Indeed, as the depth increases (from 6 to 9 hidden layers), certification performances deteriorate greatly for the baselines while ours remain stable and the computing time stays controlled. 

*(7)* We use the same heuristic as in [Lan 2022] to select the RLT cuts. Furthermore, we selected $\lfloor p n_k \rfloor$ cuts in our models to have a final number of RLT cuts similar to SDPt when having the same value of $p$. In our study on the 6x100 neural network there is not threshold below which performances degrade notably but rather a threshold above which performances improve (from 60% RLT cuts to 100% RLT cuts).




### Questions

*(1)* Fully connected neural networks on CIFAR10 have sometimes been used ([Lan, 2023]), however they do not reach a good clean accuracy (less than 40%). Their performances are thus too erratic to be certified.

*(2)* We give here the main ideas : By denoting n^k_a the number of stable active neurons on layer k, and $n^k_u$ the number of unstable neurons, when applying the chordal decomposition each block matrix $P_k$ (k=0 .. K-3) is a symmetric matrix of dimension $(1+ n^k_u  + n^k_a +1+ n^{k+1}_u  + n^{k+1}_a)$, by removing the stable neurons these sizes become $(1+ n^k_u  +1+ n^{k+1}_u )$. Now for the matrix $P_{K-2}$ that is composed of elements $z$ and of $\beta$ (with a number $|J_K|$) , the new size goes from  $(1+ n^{K-1}_u  + n^{K-1}_a +1+ n^{K}_u  + n^{K}_a + |J_K|)$ to  $(1+ n^{K-1}_u +1+ n^{K}_u + |J_K|)$. We will add this explanation in the appendix to clarify the proposition.

*(3)* The percentages represent the verification percentage obtained on the data run on. We see that our model is able to certify a significant amount of data even when the number of classes of the dataset increases. For the 50 classes and 67 classes points in our graph, we initially ran (SDP_t) only on 5 data due to the high resolution time. We ran it thoroughly and finally obtained 4% and 7% of certification respectively. 

*(4)* Our contribution is divided into three parts : (i) an exact untargeted quadratic formulation and its SDP relaxation, (ii) tightening cuts, and (iii) a stable active neuron ablation. We would like to highlight that SDP_u corresponds to the untargeted objective applied to the model  (SDP_t), for which we add additional cuts.  
We also ran our model SDPu without any RLT cuts, which is the application of the  untargeted objective to (SDP_Layer) on network 6x100, and we get a certification percentage of 70% with a mean computing time of  11s.

*(5)* Thank you for raising this point. In other approaches from literature, the choice of an adversarial target has to be made which might bias the results. However, in our paper, we run all models to certify across all targets. At the end, for each targeted method, the number of runs is equal to the number of evaluated data times the number of classes.

*(6)* We have conducted experiments to evaluate the percentage of verified data as a function of the number of stable neurons. We observe that when the proportion of active stable neurons ranges between 50% and 70%, the number of certified data points is significantly higher when pruning the active neurons. For larger proportions, pruning degrades the certification. The study of the impact magnitude is shown above (see answer to question 1).
Finally, experiments in Tab. 1 show a global trend with respect to the network depth. Indeed, as the depth increases (from 6 to 9 hidden layers), certification performances deteriorate greatly for the baselines while ours remain stable and the computing time stays controlled. 

*(7)* We use the same heuristic as in [Lan 2022] to select the RLT cuts. Furthermore, we selected $\lfloor p n_k \rfloor$ cuts in our models to have a final number of RLT cuts similar to SDPt when having the same value of $p$. In our study on the 6x100 neural network there is not threshold below which performances degrade notably but rather a threshold above which performances improve (from 60% RLT cuts to 100% RLT cuts).

# Réponse au commentaire du reviewer 2

Comment:
I'd like to thank the authors for taking the time to respond to my review. Regarding the points that were raised:

In our work, we propose an incomplete verifier i.e. we are only interested in computing a lower bound for the objective. Thus, we are comparing the performances against other incomplete verifiers. I understand that the proposed method is an incomplete verifier, however, when running e.g. GCP-CROWN with a set timeout, it is technically also an incomplete verifier (since, if time is insufficient, the verifier will not return a definitive result as to whether a given instance is robust). Even leaving this aside, in my opinion, it makes sense to compare different verifiers using the same time budget, e.g. enabling branching in bound-propagation-based verifiers. Other works also use similar timeouts for different verifiers to enable a fair comparison.

More generally, I would argue that even though SDP-based verifiers are incomplete, one "bounding step" with them is significantly more expensive than obtaining bounds with a linear-bound-based method such as GCP-CROWN. I would therefore disagree with the authors' idea of comparing SDP-based verification with single bound propagation steps in competing tools. The trade-off here is whether a given time budget should be spent on one bounding step with more precise relaxations (SDP) or whether it should be spent on many bounding steps employing much cheaper, but less precise relaxations (linear bound propagation such as GCP-CROWN).

This also ties in with the comments made regarding CNNs - I agree that no SDP verifier can handle these so far, but bound-propagation-based verifiers have no issues dealing with CNNs. Given their popularity in deep learning, not being able to verify the robustness for convolutional models is a weakness compared to existing approaches.

I'd further like to thank the authors for fixing the typos and for the revisions of the manuscript based on the points that I raised, this is much appreciated. I will take all these points into account and reevaluate my score based on them.

### Réponse


We thank the reviewer for the thoughtful comments. We posted the revised version of the manuscript, with modifications highlighted in blue. 

We agree that SDP-based relaxations provide significantly tighter bounds but at a higher computational cost, while linear bound-propagation methods such as CROWN or GCP-CROWN provide weaker bounds but at very low cost. We emphasize that Branch-and-Bound (BaB) remains the most reliable and widely used framework for certifying real-world neural networks. Modern BaB verifiers are increasingly hybrid, combining improved branching rules with multiple types of bounds, ranging from fast linear relaxations to MILP solves at deeper nodes. In such systems, the key question is not whether to use only SDP or only linear methods, but how to best allocate a fixed time budget across bounding steps of different precision and cost. Comparing the efficiency, tightness, and pruning power of individual bounding steps is therefore essential. SDP bounds could be used selectively to refine pruning when the search becomes complex or deep. Recent work integrating SDP-based bounds into BaB (Lan 2023 [6], SDP-CROWN 2025) demonstrates that these comparisons are valuable. These results indicate that SDP-based bounding steps are no longer prohibitively expensive in all settings, and that the field is actively moving toward hybrid hierarchies where SDP is used selectively.

Regarding CNNs: we fully agree that convolutional models remain extremely challenging for SDP-based verifiers, and that linear bound-propagation methods currently dominate in this domain. This is an important gap, and we acknowledge that it represents a clear practical strength of bound-propagation approaches. At the same time, recent structured SDP relaxations suggest that extensions to CNN architectures may be within reach. We do not claim that SDP is currently competitive on convolutional networks, but our point is that the methodological comparison remains relevant because SDP-based bounding will likely become increasingly inexpensive and better adapted to modern architectures.